In [14]:
# Set absolute paths
# -----------------------------
# Using absolute paths avoids FileNotFound errors
features_dir = Path("D:/VidSense/data/features")
dataset_dir = Path("D:/VidSense/data/dataset")
dataset_dir.mkdir(parents=True, exist_ok=True)  # create folder if missing

print("✔ Features folder:", features_dir)
print("✔ Dataset folder:", dataset_dir)

✔ Features folder: D:\VidSense\data\features
✔ Dataset folder: D:\VidSense\data\dataset


In [15]:
# Load all .npy feature sequences
# -----------------------------
X = []  # List to store feature sequences
y = []  # List to store numeric labels
label_map = {}  # Map label name -> number
reverse_map = {}  # Map number -> label name

# Detect all label folders (walking, running, etc.)
labels = sorted([d.name for d in features_dir.iterdir() if d.is_dir()])
print("Detected labels:", labels)

# Assign numeric labels and load each video’s features
for idx, label in enumerate(labels):
    label_map[label] = idx
    reverse_map[idx] = label

    label_dir = features_dir / label
    files = list(label_dir.glob("*.npy"))

    for f in files:
        arr = np.load(f)  # shape: (num_frames, 512)
        X.append(arr)
        y.append(idx)

print(f"✔ Total videos loaded: {len(X)}")

Detected labels: ['falling', 'running', 'sitting', 'walking']
✔ Total videos loaded: 12


In [17]:
# Pad sequences to same length
# -----------------------------
# CNN-LSTM requires all sequences to have the same number of frames
max_len = max(seq.shape[0] for seq in X)  # find max frames across all videos
feature_dim = X[0].shape[1]  # feature dimension (512)

print("Max frames in any video:", max_len)
print("Feature dimension:", feature_dim)

# Create a zero-padded array for all videos
X_padded = np.zeros((len(X), max_len, feature_dim), dtype='float32')

for i, seq in enumerate(X):
    X_padded[i, :seq.shape[0], :] = seq  # copy original features

X = X_padded
y = np.array(y, dtype='int')

print("✔ Final dataset shapes:")
print("X:", X.shape)
print("y:", y.shape)


Max frames in any video: 12
Feature dimension: 512
✔ Final dataset shapes:
X: (12, 12, 512)
y: (12,)


In [18]:
#rain/Test split
# -----------------------------
# Note: If you have very few videos per class, remove stratify=y
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print("Train shapes:", X_train.shape, y_train.shape)
print("Test shapes :", X_test.shape, y_test.shape)

Train shapes: (9, 12, 512) (9,)
Test shapes : (3, 12, 512) (3,)


In [19]:
#Save datasets safely
# -----------------------------
np.save(dataset_dir / "X_train.npy", X_train)
np.save(dataset_dir / "X_test.npy", X_test)
np.save(dataset_dir / "y_train.npy", y_train)
np.save(dataset_dir / "y_test.npy", y_test)

print("✔ Dataset saved successfully to:", dataset_dir)
print("Contents now:", list(dataset_dir.glob("*.npy")))

✔ Dataset saved successfully to: D:\VidSense\data\dataset
Contents now: [WindowsPath('D:/VidSense/data/dataset/X_test.npy'), WindowsPath('D:/VidSense/data/dataset/X_train.npy'), WindowsPath('D:/VidSense/data/dataset/y_test.npy'), WindowsPath('D:/VidSense/data/dataset/y_train.npy')]
